# 0.0 Imports

In [15]:
import pandas as pd
import gcsfs

from google.cloud import storage
from io import StringIO

## 0.1 Loading Data from CS

In [2]:
storage_client = storage.Client()

bucket = storage_client.bucket("data_pipeline_paris")
blob = bucket.blob("ParisHousing.csv")

content = blob.download_as_string()
df = pd.read_csv(StringIO(content.decode('utf-8')))

In [3]:
df.head()

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
0,75523,3,0,1,63,9373,3,8,2005,0,1,4313,9005,956,0,7,7559081.5
1,80771,39,1,1,98,39381,8,6,2015,1,0,3653,2436,128,1,2,8085989.5
2,55712,58,0,1,19,34457,6,8,2021,0,0,2937,8852,135,1,9,5574642.1
3,32316,47,0,0,6,27939,10,4,2012,0,1,659,7141,359,0,3,3232561.2
4,70429,19,1,1,90,38045,3,7,1990,1,0,8435,2429,292,1,4,7055052.0


In [4]:
df.made.unique()

array([2005, 2015, 2021, 2012, 1990, 1995, 2003, 1993, 2011, 2008, 2004,
       2018, 2009, 2013, 2016, 2017, 2000, 2019, 2020, 1994, 2014, 1998,
       2001, 1996, 1991, 1992, 1999, 1997, 2010, 2007, 2006, 2002])

# 1.0 Data Description

## 1.1 Data Dimensions

In [5]:
print("Número de linhas: {}".format(df.shape[0]))
print("Número de colunas: {}".format(df.shape[1]))

Número de linhas: 10000
Número de colunas: 17


## 1.2 Data Types

In [6]:
df.dtypes

squareMeters           int64
numberOfRooms          int64
hasYard                int64
hasPool                int64
floors                 int64
cityCode               int64
cityPartRange          int64
numPrevOwners          int64
made                   int64
isNewBuilt             int64
hasStormProtector      int64
basement               int64
attic                  int64
garage                 int64
hasStorageRoom         int64
hasGuestRoom           int64
price                float64
dtype: object

## 1.3 Check NA

In [7]:
df.isna().sum()

squareMeters         0
numberOfRooms        0
hasYard              0
hasPool              0
floors               0
cityCode             0
cityPartRange        0
numPrevOwners        0
made                 0
isNewBuilt           0
hasStormProtector    0
basement             0
attic                0
garage               0
hasStorageRoom       0
hasGuestRoom         0
price                0
dtype: int64

## 1.4 Descriptive Statistical

In [8]:
df.describe()

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.00000,10000.00000,10000.000000,10000.00000,1.000000e+04
mean,49870.13120,50.358400,0.508700,0.496800,50.276300,50225.486100,5.510100,5.521700,2005.48850,0.499100,0.499900,5033.103900,5028.01060,553.12120,0.503000,4.99460,4.993448e+06
std,28774.37535,28.816696,0.499949,0.500015,28.889171,29006.675799,2.872024,2.856667,9.30809,0.500024,0.500025,2876.729545,2894.33221,262.05017,0.500016,3.17641,2.877424e+06
min,89.00000,1.000000,0.000000,0.000000,1.000000,3.000000,1.000000,1.000000,1990.00000,0.000000,0.000000,0.000000,1.00000,100.00000,0.000000,0.00000,1.031350e+04
25%,25098.50000,25.000000,0.000000,0.000000,25.000000,24693.750000,3.000000,3.000000,1997.00000,0.000000,0.000000,2559.750000,2512.00000,327.75000,0.000000,2.00000,2.516402e+06
50%,50105.50000,50.000000,1.000000,0.000000,50.000000,50693.000000,5.000000,5.000000,2005.50000,0.000000,0.000000,5092.500000,5045.00000,554.00000,1.000000,5.00000,5.016180e+06
75%,74609.75000,75.000000,1.000000,1.000000,76.000000,75683.250000,8.000000,8.000000,2014.00000,1.000000,1.000000,7511.250000,7540.50000,777.25000,1.000000,8.00000,7.469092e+06
max,99999.00000,100.000000,1.000000,1.000000,100.000000,99953.000000,10.000000,10.000000,2021.00000,1.000000,1.000000,10000.000000,10000.00000,1000.00000,1.000000,10.00000,1.000677e+07


# 2.0 Feature Engineering

In [9]:
bins = [0, 2000, 2010, 2025]  
labels = ['before_2000', '2000_2010', 'after_2010']
df['year_category'] = pd.cut(df['made'], bins=bins, labels=labels)

df['before_2000'] = (df['year_category'] == 'before_2000').astype(int)
df['2000_2010'] = (df['year_category'] == '2000_2010').astype(int)
df['after_2010'] = (df['year_category'] == 'after_2010').astype(int)

df.drop('year_category', axis=1, inplace=True)
df.drop('made', axis=1, inplace=True)

In [10]:
df.head()

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price,before_2000,2000_2010,after_2010
0,75523,3,0,1,63,9373,3,8,0,1,4313,9005,956,0,7,7559081.5,0,1,0
1,80771,39,1,1,98,39381,8,6,1,0,3653,2436,128,1,2,8085989.5,0,0,1
2,55712,58,0,1,19,34457,6,8,0,0,2937,8852,135,1,9,5574642.1,0,0,1
3,32316,47,0,0,6,27939,10,4,0,1,659,7141,359,0,3,3232561.2,0,0,1
4,70429,19,1,1,90,38045,3,7,1,0,8435,2429,292,1,4,7055052.0,1,0,0


# 3.0 Save in CS Bucket

In [16]:
bucket_name = 'data_pipeline_paris'
file_name = 'modified_data.csv'

df.to_csv(f'gs://{bucket_name}/{file_name}', index=False)